In [ ]:
import os
import glob
import json
import extract, index, prepare
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
llm = GPT4All(  model="../models/mistral-7b-instruct-v0.1.Q4_0.gguf",
                backend="gptj",
                verbose=True,
                temp=0.3,
                top_p=0.6,
                repeat_penalty=1.18)

In [ ]:
extract.extract_tex_files(query="safety in AI", target_dir="papers_example", max_results=10)

filenames, errors = prepare.files_to_plain_text("papers_example", recursive=True)

with open('out.txt', 'w') as f:
    for line in filenames:
        f.write(f"{line}\n")

docs = []
    

for filename in filenames:
    with open(filename) as f:
        text = f.read()
        docs.extend(prepare.split_to_sentences(text))

In [ ]:
db_name = "simple-rag"

In [ ]:

for article in os.listdir("./papers_example"):
    path = "./papers_example/" + article
    text = []
    for document in os.listdir(path):
        if document.endswith("txt"):
            with open(f"{path}/{document}") as f:
                text.append(f.read())
        if document.endswith("metadata.json"):
            with open(f"{path}/{document}") as f:
                metadata = json.load(f)
    print("**********************")
    print(article)
    print(metadata)
    print(text)

In [ ]:
index.populate_and_prepare_db(db_name, docs)

In [ ]:
query = "What is Circumstantial Inference?" 

In [ ]:
# results after removing duplicates
results = index.search_for_documents(query, db_name, k=10)

In [ ]:
for i in range(len(results)):
    print('-' * 10, "Data " + str(i+1), '-' * 10)
    print(results[i].page_content, '\n')

In [ ]:
[r for r in results if "38" in r.page_content]

In [ ]:
prompt_template = ChatPromptTemplate.from_template("Using information enclosed in {results} answer query {query}.")
prompt = prompt_template.format(results=results, query=query)

response = llm(prompt)

print(response)

In [ ]:
response

In [ ]:
llm("tell me a joke")